# Configuration Azure ML

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

## 1. Infos

In [1]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [2]:
import datetime
maintenant = datetime.datetime.now()
print('Date :', maintenant)

Date : 2020-04-30 12:55:46.685923


In [3]:
from azureml.core import Workspace, Environment

## 2. Installation librairie AzureML service

Note : Le kernel Python 3.6 Azure ML contient déjà Azure ML.


> Release notes Azure ML service : https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes

> Installation : https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py

### 2.1 Pour installer la première fois le SDK

In [4]:
#pip install azureml-sdk

> Redémarrer ensuite le kernel Jupyter

### 2.2 Pour mettre à jour Azure ML service

In [5]:
#pip install --upgrade azureml-sdk

Penser à mettre à jour les extras et ensuite à relancer le kernel Jupyter

In [6]:
#pip install --upgrade azureml-sdk[automl,notebooks,explain,accel-models,services,tensorboard]

## 3. Version Azure ML installée

In [7]:
import azureml.core
print("Version Azure ML =",azureml.core.VERSION)

Version Azure ML = 1.4.0


In [8]:
pip list | grep azureml

azureml-accel-models                  1.4.0        
azureml-automl-core                   1.2.0        
azureml-automl-runtime                1.2.0        
azureml-contrib-gbdt                  1.4.0        
azureml-contrib-interpret             1.2.0        
azureml-contrib-notebook              1.2.0        
azureml-contrib-pipeline-steps        1.2.0        
azureml-contrib-reinforcementlearning 1.2.0        
azureml-contrib-server                1.2.0        
azureml-contrib-services              1.2.0        
azureml-core                          1.4.0.post1  
azureml-datadrift                     1.4.0        
azureml-dataprep                      1.4.6        
azureml-dataprep-native               14.1.0       
azureml-defaults                      1.2.0        
azureml-explain-model                 1.2.0        
azureml-interpret                     1.2.0        
azureml-mlflow                        1.2.0        
azureml-model-management-sdk          1.0.1b6.post1
azureml-open

> Versions Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/azure-machine-learning-release-notes

## 4. Workspace Azure ML

In [9]:
import os
subscription_id = os.environ.get("SUBSCRIPTION_ID", "AREMPLACER") #ID soubscription Azure
resource_group = os.environ.get("RESOURCE_GROUP", "workshopAML2020-rg") #Resource group
workspace_name = os.environ.get("WORKSPACE_NAME", "workshopAML2020") #Nom workspace

In [10]:
from azureml.core import Workspace

try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print("Le workspace Azure ML service a été trouvé : OK")
except:
   print("Le workspace Azure ML service n'a pas été trouvé")

Le workspace Azure ML service a été trouvé : OK


In [16]:
# Lecture du fichier de config
ws = Workspace.from_config()
print('Workspace : ' + ws.name, 
      'Région : ' + ws.location, 
      'Groupe de ressources : ' + ws.resource_group, sep='\n')

Workspace : workshopAML2020
Région : westeurope
Groupe de ressources : workshopAML2020-rg


In [17]:
# Infos
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourceGroups/workshopAML2020-rg/providers/Microsoft.MachineLearningServices/workspaces/workshopAML2020',
 'name': 'workshopAML2020',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Enterprise',
 'workspaceid': '1696467d-5136-4ed9-9f78-4c69eaff7896',
 'description': '',
 'friendlyName': '',
 'creationTime': '2020-03-19T12:49:48.1312139+00:00',
 'containerRegistry': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourceGroups/workshopAML2020-rg/providers/Microsoft.ContainerRegistry/registries/workshopaml2d644fd00',
 'keyVault': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/workshopaml2020-rg/providers/microsoft.keyvault/vaults/workshopaml2027384601059',
 'applicationInsights': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/workshopaml2020-rg/providers/microsoft.insights/components/workshopaml2029456289985',
 'identityPri

In [19]:
from azureml.core import ComputeTarget, Datastore, Dataset

print("- Compute Targets :")
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("\t", compute.name, ':', compute.type)
    
print("- Datastores :")
for datastore_name in ws.datastores:
    datastore = Datastore.get(ws, datastore_name)
    print("\t", datastore.name, ':', datastore.datastore_type)
    
print("- Datasets :")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name)

- Compute Targets :
	 instance : ComputeInstance
	 cpu-cluster : AmlCompute
- Datastores :
	 teststorageserge : AzureBlob
	 azureml_globaldatasets : AzureBlob
	 workspaceblobstore : AzureBlob
	 workspacefilestore : AzureFile
- Datasets :
	 target
	 dataset
	 Iris
	 mnist dataset
	 diabetes dataset


# (Optionnel)

## 5. Gestion des environnements
> Documentation : https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-use-environments

### 5.1 Environnements existants

### Liste des environnements

In [8]:
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Nom :",env)
        print()
        print("Packages :", envs[env].python.conda_dependencies.serialize_to_string())

Nom : AzureML-Tutorial

Packages : channels:
- anaconda
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.3.0
  - azureml-defaults==1.3.0
  - azureml-telemetry==1.3.0
  - azureml-train-restclients-hyperdrive==1.3.0
  - azureml-train-core==1.3.0
  - azureml-widgets==1.3.0
  - azureml-pipeline-core==1.3.0
  - azureml-pipeline-steps==1.3.0
  - azureml-opendatasets==1.3.0
  - azureml-automl-core==1.3.0
  - azureml-automl-runtime==1.3.0
  - azureml-train-automl-client==1.3.0
  - azureml-train-automl-runtime==1.3.0
  - azureml-train-automl==1.3.0
  - azureml-train==1.3.0
  - azureml-sdk==1.3.0
  - azureml-interpret==1.3.0
  - azureml-tensorboard==1.3.0
  - azureml-mlflow==1.3.0
  - mlflow
  - sklearn-pandas
- pandas
- numpy
- tqdm
- scikit-learn
- matplotlib
name: azureml_1a1da8eed182746d60e561b734a086c3

Nom : AzureML-Minimal

Packages : channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.3.0
  - azureml-defaults==1.3.0
name: azureml_f851a99ad

In [9]:
# Info environnement Azure ML Minimal

ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="AzureML-Minimal")

In [10]:
# Info environnement Azure ML Minimal

env = Environment.get(workspace=ws, name="AzureML-Minimal")
print("Nom : ",env)
print()
print("packages : ", env.python.conda_dependencies.serialize_to_string())

Nom :  Environment(Name: AzureML-Minimal,
Version: 20)

packages :  channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.3.0
  - azureml-defaults==1.3.0
name: azureml_f851a99ad48fd8bf6717f441e3edf254



In [11]:
# AzureML-AutoML-DNN

env = Environment.get(workspace=ws, name="AzureML-AutoML-DNN")
print("Nom : ",env)
print()
print("Packages : ", env.python.conda_dependencies.serialize_to_string())

Nom :  Environment(Name: AzureML-AutoML-DNN,
Version: 9)

Packages :  channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.3.0
  - azureml-pipeline-core==1.3.0
  - azureml-telemetry==1.3.0
  - azureml-defaults==1.3.0
  - azureml-interpret==1.3.0
  - azureml-explain-model==1.3.0
  - azureml-automl-core==1.3.0
  - azureml-automl-runtime==1.3.0
  - azureml-train-automl-client==1.3.0
  - azureml-train-automl-runtime==1.3.0
  - inference-schema
  - pytorch-transformers==1.0.0
  - spacy==2.1.8
  - https://aka.ms/automl-resources/packages/en_core_web_sm-2.1.0.tar.gz
- pandas>=0.21.0,<=0.23.4
- numpy>=1.16.0,<=1.16.2
- py-xgboost<=0.90
- fbprophet==0.5
- setuptools-git
- pytorch=1.1.0
- cudatoolkit=9.0
name: azureml_9ecdd5c1fa1115e7a59fbd65be76793b



### 5.2 Création d'un environnement

In [41]:
from azureml.core.environment import CondaDependencies

monenvironnement = Environment(name="monenvironnement")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")

In [58]:
conda_dep.add_pip_package("scikit-learn==0.22.2")
myenv.python.conda_dependencies=conda_dep

In [59]:
monenvironnement.environment_variables = {"MESSAGE":"Mon environnement scikit"}

In [60]:
from azureml.core import ScriptRunConfig, Experiment

myexp = Experiment(workspace=ws, name = "Exemple0-Environnement")

### Création fichier python

In [65]:
%%writefile exemplepython.py

import random
for n in range(10):
  print(random.randint(1,101))


Overwriting exemplepython.py


In [66]:
!ls exemplepython.py -l

-rwxrwxrwx 1 root root 66 Apr 23 08:43 exemplepython.py


In [67]:
tags = {"type": "test" , "langage" : "Python"}

In [68]:
runconfig = ScriptRunConfig(source_directory=".", script="exemplepython.py")
runconfig.run_config.target = "local"
runconfig.run_config.environment = monenvironnement

run = myexp.submit(config=runconfig, tags=tags)
run.wait_for_completion(show_output=True)

RunId: Exemple0-Environnement_1587631409_f73f2431
Web View: https://ml.azure.com/experiments/Exemple0-Environnement/runs/Exemple0-Environnement_1587631409_f73f2431?wsid=/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/workshopAML2020-rg/workspaces/workshopAML2020

Execution Summary
RunId: Exemple0-Environnement_1587631409_f73f2431
Web View: https://ml.azure.com/experiments/Exemple0-Environnement/runs/Exemple0-Environnement_1587631409_f73f2431?wsid=/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/workshopAML2020-rg/workspaces/workshopAML2020



{'runId': 'Exemple0-Environnement_1587631409_f73f2431',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-04-23T08:44:01.590087Z',
 'endTimeUtc': '2020-04-23T08:44:05.944878Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'e6dea94e-75a4-4ca2-9f23-ea9fd3917bc5',
  'azureml.git.repository_uri': 'https://github.com/retkowsky/WorkshopAML2020',
  'mlflow.source.git.repoURL': 'https://github.com/retkowsky/WorkshopAML2020',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'eb05ad565a41b5121d26c6fda4b1c6398a9243d7',
  'mlflow.source.git.commit': 'eb05ad565a41b5121d26c6fda4b1c6398a9243d7',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'runDefinition': {'script': 'exemplepython.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'jobName': None

In [70]:
run.get_environment()

{
    "name": "monenvironnement",
    "version": "Autosave_2020-04-23T08:36:25Z_97d19ce1",
    "environmentVariables": {
        "MESSAGE": "Mon environnement scikit"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
            ],
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
                        "azureml-defaults"
                    ]
                }
            ],
            "name": "azureml_da3e97fcb51801118b8e80207f3e01ad"
        }
    },
    "docker": {
        "enabled": false,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseDockerfile": null,
        "sharedVolumes": true,
        "shmSize": null,
        "arguments": [],


In [71]:
# Enregistrement environnement
monenvironnement.register(workspace=ws)

{
    "name": "monenvironnement",
    "version": "2",
    "environmentVariables": {
        "MESSAGE": "Mon environnement scikit"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
            ],
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
                        "azureml-defaults"
                    ]
                }
            ],
            "name": "azureml_da3e97fcb51801118b8e80207f3e01ad"
        }
    },
    "docker": {
        "enabled": false,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseDockerfile": null,
        "sharedVolumes": true,
        "shmSize": null,
        "arguments": [],
        "baseImageRegistry": {
      

### Liste des environnements

In [72]:
# Liste - on voit bien la création de l'environnement
for name,env in ws.environments.items():
    print("Nom {} \t - version {}".format(name,env.version))

Nom myenv 	 - version 2
Nom diabetes-experiment-env 	 - version 2
Nom monenvironnement 	 - version 2
Nom AzureML-Tutorial 	 - version 25
Nom AzureML-Minimal 	 - version 20
Nom AzureML-Chainer-5.1.0-GPU 	 - version 19
Nom AzureML-PyTorch-1.2-CPU 	 - version 19
Nom AzureML-TensorFlow-1.12-CPU 	 - version 18
Nom AzureML-TensorFlow-1.13-CPU 	 - version 19
Nom AzureML-PyTorch-1.1-CPU 	 - version 19
Nom AzureML-TensorFlow-1.10-CPU 	 - version 19
Nom AzureML-PyTorch-1.0-GPU 	 - version 19
Nom AzureML-TensorFlow-1.12-GPU 	 - version 19
Nom AzureML-TensorFlow-1.13-GPU 	 - version 19
Nom AzureML-Chainer-5.1.0-CPU 	 - version 19
Nom AzureML-PyTorch-1.0-CPU 	 - version 19
Nom AzureML-Scikit-learn-0.20.3 	 - version 19
Nom AzureML-PyTorch-1.2-GPU 	 - version 19
Nom AzureML-PyTorch-1.1-GPU 	 - version 19
Nom AzureML-TensorFlow-1.10-GPU 	 - version 19
Nom AzureML-PyTorch-1.3-GPU 	 - version 17
Nom AzureML-TensorFlow-2.0-CPU 	 - version 17
Nom AzureML-PyTorch-1.3-CPU 	 - version 16
Nom AzureML-TensorF

In [73]:
restored_environment = Environment.get(workspace=ws,name="monenvironnement",version="1")

print("Visualisation environnement :")
restored_environment

Visualisation environnement :


{
    "name": "monenvironnement",
    "version": "1",
    "environmentVariables": {
        "MESSAGE": "Mon environnement !"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
            ],
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
                        "azureml-defaults"
                    ]
                }
            ],
            "name": "azureml_da3e97fcb51801118b8e80207f3e01ad"
        }
    },
    "docker": {
        "enabled": false,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseDockerfile": null,
        "sharedVolumes": true,
        "shmSize": null,
        "arguments": [],
        "baseImageRegistry": {
           

## 6. Azure ML Studio

https://ml.azure.com/

<img src="https://github.com/retkowsky/images/blob/master/LandingPage.jpg?raw=true">

## 7. Azure ML CLI (Command Line Interface)

>Documentation Azure ML CLI :
https://docs.microsoft.com/en-us/azure/machine-learning/reference-azure-machine-learning-cli

>Prérequis : Azure CLI
Pour installer Azure CLI sur Windows :
https://docs.microsoft.com/en-us/cli/azure/install-azure-cli-windows?view=azure-cli-latest
<br>

**. Installation Azure ML CLI<br>**
az extension add -n azure-cli-ml

**. Check<br>**
az ml -h<br>

az --version

**. Création ressource groupe<br>**
az group create -n mltestRG -l westeurope

**. Création workspace Azure ML<br>**
az ml workspace create -w monworkspace -g mltestRG

**. Attach workspace Azure ML<br>**
az ml folder attach -w monworkspace -g mltestRG

**. Création compute server mycomputesrv<br>**
az ml computetarget create  amlcompute -n mysrv1 --min-nodes 0 --max-nodes 2   -s STANDARD_D2_V2<br>

az ml computetarget create  amlcompute -n mysrv2 --min-nodes 1 --max-nodes 2   -s STANDARD_D3_V2

**. Liste des compute servers<br>**
az ml computetarget list

**. Liste experimentations<br>**
az ml experiment list

# Documentation

- Documentation :<br> https://docs.microsoft.com/en-us/azure/machine-learning/

- Nouveautés versions Azure ML service: <br>
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes?view=azure-ml-py

- What is Azure ML service SDK?<br>
https://docs.microsoft.com/en-us/python/api/overview/azure/ml/intro?view=azure-ml-py

- Azure ML service CLI<br>
https://docs.microsoft.com/en-us/cli/azure/ext/azure-cli-ml/ml?view=azure-cli-latest

- Azure ML Visual Interface<br>
Algorithm & module reference overview<br>
https://docs.microsoft.com/en-us/azure/machine-learning/algorithm-module-reference/module-reference


<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">